# 🤗 End-to-end distilabel example with Inference Endpoints and Notus

In [1]:
import os
import time
from typing import Dict

import argilla as rg

from distilabel.llm import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline, pipeline
from distilabel.tasks import Llama2TextGenerationTask, SelfInstructTask, Prompt

from datasets import Dataset
from haystack.nodes import PDFToTextConverter, PreProcessor

In [2]:
os.environ["HF_TOKEN"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["ARGILLA_API_URL"] = "https://argilla-ultrafeedback-curator.hf.space"
os.environ["ARGILLA_API_KEY"] = "admin.apikey"

In [3]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
rg.init(
    api_url="https://ignacioct-argilla.hf.space",
    api_key="owner.apikey",
    workspace="admin"
)

/Users/ignacio/Documents/recognai/argilla/src/argilla/client/client.py:182: UserWarning: You're connecting to Argilla Server 1.19.0 using a different client version (1.20.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


## Setting up an inference endpoint with Notus

To kickstart this tutorial, let's see how to set up and endpoint for our Notus model. A HuggingFace endpoint is a service provided by HuggingFace that allows you to deploy and host your machine learning models for inference. This way, we'll have faster inference times, as these models will not run in our personal machines, but in HuggingFace servers. The endpoint of choice has a [Notus 7B instance](https://ui.endpoints.huggingface.co/argilla/endpoints/aws-notus-7b-v1-4052) running.

Let's see a quick example of how to use an inference endpoint. We have prepared an easy `Llama2QuestionAnsweringTask` to ask question to the model, in a very similar way as we talk with the LLMs using chatbots.

In [4]:
class Llama2QuestionAnsweringTask(Llama2TextGenerationTask):
    def generate_prompt(self, question: str) -> str:
        return Prompt(
            system_prompt=self.system_prompt,
            formatted_prompt=question,
        ).format_as("llama2")  # type: ignore

    def parse_output(self, output: str) -> Dict[str, str]:
        return {"answer": output.strip()}

    def input_args_names(self) -> list[str]:
        return ["question"]

    def output_args_names(self) -> list[str]:
        return ["answer"]

Once this class is ready, we have to instantiate an `InferenceEndpointsLLM` object, and pass as parameters the HF Inference Endpoint name and the HF namespace. One very convenient way to do so is through environment variables.

In [5]:
os.environ["HF_INFERENCE_ENDPOINT_NAME"] = "aws-notus-7b-v1-4052"
os.environ["HF_NAMESPACE"] = "argilla"

A HuggingFace Token is also required to use HuggingFace's services.

In [6]:
llm = InferenceEndpointsLLM(
    endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token=os.getenv("HF_TOKEN") or None,
    task=Llama2QuestionAnsweringTask(),
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/ignacio/.local/lib/python3.11/site-packages/huggingface_hub/utils/_errors.py:270 in       │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   267 │   </Tip>                                                                                 │
│   268 │   """                                                                                    │
│   269 │   try:                                                                                   │
│ ❱ 270 │   │   response.raise_for_status()                                                        │
│   271 │   except HTTPError as e:                                                                 │
│   272 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   273                                                                                            │
│                                                                                                  │
│ /Users/ignacio/.local/lib/python3.11/site-packages/requests/models.py:1021 in raise_for_status   │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: 
https://api.endpoints.huggingface.cloud/v2/endpoint/argilla/aws-notus-7b-v1-4052

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 llm = InferenceEndpointsLLM(                                                                 │
│   2 │   endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore                   │
│   3 │   endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore                            │
│   4 │   token=os.getenv("HF_TOKEN") or None,                                                     │
│                                                                                                  │
│ /Users/ignacio/.local/lib/python3.11/site-packages/distilabel/llm/huggingface/inference_endpoint │
│ s.py:124 in __init__                                                                             │
│                                                                                                  │
│   121 │   │   self.top_p = top_p                                                                 │
│   122 │   │   self.typical_p = typical_p                                                         │
│   123 │   │                                                                                      │
│ ❱ 124 │   │   self.inference_endpoint = get_inference_endpoint(                                  │
│   125 │   │   │   name=endpoint_name, namespace=endpoint_nam

The `llm` is an object of the `InferenceEndpointsLLM` class, and through it we can start generating answers to question using the `llm.generate()` method.

In [7]:
generation = llm.generate([{"question": "What's the second most populated city in Denmark?"}])
generation[0][0]["parsed_output"]["answer"]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 generation = llm.generate([{"question": "What's the second most populated city in Denmar     │
│   2 generation[0][0]["parsed_output"]["answer"]                                                  │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'llm' is not defined

The endpoint is working! We now can do inference through the Inference Endpoint.

## Downloading input dataset for 

## Generating instructions with SelfInstructTask

With out Inference Endpoint up and running, we should be able to generate instructions with distilabel. These instructions, made by the LLM through our endpoint, will form an instruction dataset.

In [8]:
!wget https://artificialintelligenceact.eu/wp-content/uploads/2021/08/The-AI-Act.pdf

--2023-12-18 10:14:00--  https://artificialintelligenceact.eu/wp-content/uploads/2021/08/The-AI-Act.pdf
Resolving artificialintelligenceact.eu (artificialintelligenceact.eu)... 173.255.227.216
Connecting to artificialintelligenceact.eu (artificialintelligenceact.eu)|173.255.227.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1351521 (1,3M) [application/pdf]
Saving to: ‘The-AI-Act.pdf.1’

The-AI-Act.pdf.1    100%[===================>]   1,29M  1,17MB/s    in 1,1s    

2023-12-18 10:14:02 (1,17 MB/s) - ‘The-AI-Act.pdf.1’ saved [1351521/1351521]



In [9]:
converter = PDFToTextConverter(
    remove_numeric_tables=True,
    valid_languages=["en"]
)

doc = converter.convert(file_path="The-AI-Act.pdf", meta=None)[0]

pdftotext version 4.04 [www.xpdfreader.com]
Copyright 1996-2022 Glyph & Cog, LLC


In [10]:
doc

<Document: {'content': 'EN EN\nEUROPEAN\nCOMMISSION\nProposal for a\nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE\n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION\nLEGISLATIVE ACTS\x0cEN\nEXPLANATORY MEMORANDUM\n1. CONTEXT OF THE PROPOSAL\n1.1. Reasons for and objectives of the proposal\nThis explanatory memorandum accompanies the proposal for a Regulation laying down\nharmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence\n(AI) is a fast evolving family of technologies that can bring a wide array of economic and\nsocietal benefits across the entire spectrum of industries and social activities. By improving\nprediction, optimising operations and resource allocation, and personalising service delivery,\nthe use of artificial intelligence can support socially and environmentally beneficial outcomes\nand provide key competitive advantages to companies and the European 

In [11]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=150,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process([doc])
print(f"n_docs_input: 1\nn_docs_output: {len(docs)}")


Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

[12/18/23 10:14:03] WARNING  WARNING:haystack.nodes.preprocessor.preprocessor:We found one or   ]8;id=888426;file:///Users/ignacio/.local/lib/python3.11/site-packages/haystack/nodes/preprocessor/preprocessor.py\preprocessor.py]8;;\:]8;id=709496;file:///Users/ignacio/.local/lib/python3.11/site-packages/haystack/nodes/preprocessor/preprocessor.py#494\494]8;;\
                             more sentences whose word count is higher than the split length.                      

Preprocessing: 100%|██████████| 1/1 [00:00<00:00,  3.27docs/s]

n_docs_input: 1
n_docs_output: 355


In [12]:
docs[0].content

'EN EN\nEUROPEAN\nCOMMISSION\nProposal for a\nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE\n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION\nLEGISLATIVE ACTS\x0cEN\nEXPLANATORY MEMORANDUM\n1. CONTEXT OF THE PROPOSAL\n1.1. Reasons for and objectives of the proposal\nThis explanatory memorandum accompanies the proposal for a Regulation laying down\nharmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence\n(AI) is a fast evolving family of technologies that can bring a wide array of economic and\nsocietal benefits across the entire spectrum of industries and social activities. By improving\nprediction, optimising operations and resource allocation, and personalising service delivery,\nthe use of artificial intelligence can support socially and environmentally beneficial outcomes\nand provide key competitive advantages to companies and the European economy. '

In [13]:
inputs = [doc.content for doc in docs]
inputs

['EN EN\nEUROPEAN\nCOMMISSION\nProposal for a\nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE\n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION\nLEGISLATIVE ACTS\x0cEN\nEXPLANATORY MEMORANDUM\n1. CONTEXT OF THE PROPOSAL\n1.1. Reasons for and objectives of the proposal\nThis explanatory memorandum accompanies the proposal for a Regulation laying down\nharmonised rules on artificial intelligence (Artificial Intelligence Act). Artificial Intelligence\n(AI) is a fast evolving family of technologies that can bring a wide array of economic and\nsocietal benefits across the entire spectrum of industries and social activities. By improving\nprediction, optimising operations and resource allocation, and personalising service delivery,\nthe use of artificial intelligence can support socially and environmentally beneficial outcomes\nand provide key competitive advantages to companies and the European economy. ',
 'Such\nac

In [14]:
instructions_dataset = Dataset.from_dict({
    "input": inputs[0:50]
})

In [15]:
instructions_dataset

Dataset({
    features: ['input'],
    num_rows: 50
})

In [16]:
instructions_task = SelfInstructTask(
    application_description="A assistant that can answer questions about the AI Act made by the European Union."
)

Let's now define a generator, passing the `SelfInstructTask` object, and create a `Pipeline` object.

In [17]:
instructions_generator = InferenceEndpointsLLM(
    endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
    endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore
    token=os.getenv("HF_TOKEN") or None,
    task=instructions_task,
)

instructions_pipeline = Pipeline(
    generator=instructions_generator
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/ignacio/.local/lib/python3.11/site-packages/huggingface_hub/utils/_errors.py:270 in       │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   267 │   </Tip>                                                                                 │
│   268 │   """                                                                                    │
│   269 │   try:                                                                                   │
│ ❱ 270 │   │   response.raise_for_status()                                                        │
│   271 │   except HTTPError as e:                                                                 │
│   272 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   273                                                                                            │
│                                                                                                  │
│ /Users/ignacio/.local/lib/python3.11/site-packages/requests/models.py:1021 in raise_for_status   │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: 
https://api.endpoints.huggingface.cloud/v2/endpoint/argilla/aws-notus-7b-v1-4052

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 instructions_generator = InferenceEndpointsLLM(                                             │
│    2 │   endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore                  │
│    3 │   endpoint_namespace=os.getenv("HF_NAMESPACE"),  # type: ignore                           │
│    4 │   token=os.getenv("HF_TOKEN") or None,                                                    │
│                                                                                                  │
│ /Users/ignacio/.local/lib/python3.11/site-packages/distilabel/llm/huggingface/inference_endpoint │
│ s.py:124 in __init__                                                                             │
│                                                                                                  │
│   121 │   │   self.top_p = top_p                                                                 │
│   122 │   │   self.typical_p = typical_p                                                         │
│   123 │   │                                                                                      │
│ ❱ 124 │   │   self.inference_endpoint = get_inference_endpoint(                                  │
│   125 │   │   │   name=endpoint_name, namespace=endpoint_nam

Our pipeline is ready to be used to generate instructions. Let's do it!

In [18]:
generated_instructions = instructions_pipeline.generate(dataset=instructions_dataset, num_generations=2, batch_size=8)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 generated_instructions = instructions_pipeline.generate(dataset=instructions_dataset, nu     │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'instructions_pipeline' is not defined

Our pipeline has succesfully generated instructions given the topics and the behaviour passed as input. Let's gather all those instructions and see how the look.

In [19]:
instructions = []
for generations in generated_instructions["generations"]:
    for generation in generations:
        instructions.extend(generation)

print(f"Number of generated instructions: {len(instructions)}")

for instruction in instructions[:5]:
    print(instruction)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 instructions = []                                                                           │
│ ❱  2 for generations in generated_instructions["generations"]:                                   │
│    3 │   for generation in generations:                                                          │
│    4 │   │   instructions.extend(generation)                                                     │
│    5                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'generated_instructions' is not defined

These instruction are really usefull in our story-making task, as we can start building a fictional world by just answering them.

## Generate a Preference Dataset using an Ultrafeedback text quality task.

Another possibility with Distilabel is to create a Preference Dataset through an Ultrafeedback text quality task. It's a type of task used in NLP to evaluate the quality of text generated. Our goal is to provide detailed feedback on the quality of the generated text, beyond just a binary label. 

Our `pipeline()` method allows us to create a `Pipeline` instance with the provided LLMs for a given task, which is useful whenever you want to use a pre-defined or custom `Pipeline` for a given task. We will specify our task and subtask, the generator we want to use (in this case, one based in a Llama2 Text Generator Task) and our OpenAI API key.

In [20]:
preference_pipeline = pipeline(
    "preference",
    "text-quality",
    generator=InferenceEndpointsLLM(
        endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore
        endpoint_namespace=os.getenv("HF_NAMESPACE", None),
        task=Llama2TextGenerationTask(),
        max_new_tokens=256,
        num_threads=2,
        temperature=0.3,
    ),
    max_new_tokens=256,
    num_threads=2,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/ignacio/.local/lib/python3.11/site-packages/huggingface_hub/utils/_errors.py:270 in       │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   267 │   </Tip>                                                                                 │
│   268 │   """                                                                                    │
│   269 │   try:                                                                                   │
│ ❱ 270 │   │   response.raise_for_status()                                                        │
│   271 │   except HTTPError as e:                                                                 │
│   272 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   273                                                                                            │
│                                                                                                  │
│ /Users/ignacio/.local/lib/python3.11/site-packages/requests/models.py:1021 in raise_for_status   │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: 
https://api.endpoints.huggingface.cloud/v2/endpoint/argilla/aws-notus-7b-v1-4052

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 preference_pipeline = pipeline(                                                             │
│    2 │   "preference",                                                                           │
│    3 │   "text-quality",                                                                         │
│ ❱  4 │   generator=InferenceEndpointsLLM(                                                        │
│    5 │   │   endpoint_name=os.getenv("HF_INFERENCE_ENDPOINT_NAME"),  # type: ignore              │
│    6 │   │   endpoint_namespace=os.getenv("HF_NAMESPACE", None),                                 │
│    7 │   │   task=Llama2TextGenerationTask(),                                                    │
│                                                                                                  │
│ /Users/ignacio/.local/lib/python3.11/site-packages/distilabel/llm/huggingface/inference_endpoint │
│ s.py:124 in __init__                                                                             │
│                                                                                                  │
│   121 │   │   self.top_p = top_p                                                                 │
│   122 │   │   self.typical_p = typical_p                    

In [21]:
instructions_dataset = Dataset.from_dict({
    "input": instructions[0:50]
})

Now, let's build a dataset by using the pipeline we just created, and the topics from which our instructions were generated. They are still valid, as we want to create a preference dataset still focus on writing characters and stories.

In [22]:
preference_dataset = preference_pipeline.generate(
    instructions_dataset,  # type: ignore
    num_generations=2,
    batch_size=8,
    enable_checkpoints=True,
    display_progress_bar=True,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 preference_dataset = preference_pipeline.generate(                                           │
│   2 │   instructions_dataset,  # type: ignore                                                    │
│   3 │   num_generations=2,                                                                       │
│   4 │   batch_size=8,                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'preference_pipeline' is not defined

Let's take a look at an instance of the preference dataset

In [23]:
preference_dataset[0]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 preference_dataset[0]                                                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'preference_dataset' is not defined

## Setting up an Argilla HF Space to upload the resulting dataset.

In [24]:
rg.init(
    api_url=os.getenv("ARGILLA_API_URL"), api_key=os.getenv("ARGILLA_API_KEY")
)

/Users/ignacio/Documents/recognai/argilla/src/argilla/client/client.py:165: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
/Users/ignacio/Documents/recognai/argilla/src/argilla/client/client.py:182: UserWarning: You're connecting to Argilla Server 1.19.0 using a different client version (1.20.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [25]:
# Uploading the Preference Dataset
preference_rg_dataset = preference_dataset.to_argilla()
preference_rg_dataset.push_to_argilla(name=f"notus_AI_preference")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Uploading the Preference Dataset                                                           │
│ ❱ 2 preference_rg_dataset = preference_dataset.to_argilla()                                      │
│   3 preference_rg_dataset.push_to_argilla(name=f"notus_AI_preference")                           │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'preference_dataset' is not defined